## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df['APPLICATION_TYPE'].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 1000
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(application_counts[application_counts < cutoff].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at Affiliation value counts for binning
print(application_df['AFFILIATION'].value_counts())

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64


In [7]:
# Group low-frequency AFFILIATION categories
low_freq_categories = ['Family/Parent', 'National', 'Regional', 'Other']
application_df['AFFILIATION'] = application_df['AFFILIATION'].apply(lambda x: 'Other' if x in low_freq_categories else x)

# Check the new value counts
print(application_df['AFFILIATION'].value_counts())

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64


In [8]:
# Look at CLASSIFICATION value counts for binning
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1820        1
C2380        1
C4200        1
C1248        1
C2500        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C3200        2
C1234        2
C1267        2
C1246        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1000
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < cutoff].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Look at USE_CASE
print(application_df['USE_CASE'].value_counts())

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64


In [12]:
# Group low-frequency USE_CASE categories
low_freq_categories = ['CommunityServ', 'Heathcare', 'Other']
application_df['USE_CASE'] = application_df['USE_CASE'].apply(lambda x: 'Other' if x in low_freq_categories else x)

# Check the new value counts
print(application_df['USE_CASE'].value_counts())

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64


In [13]:
# Look at ORGANIZATION
print(application_df['ORGANIZATION'].value_counts())

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64


In [14]:
# Group low-frequency ORGANIZATION categories
low_freq_categories = ['Co-operative', 'Corporation']
application_df['ORGANIZATION'] = application_df['ORGANIZATION'].apply(lambda x: 'Other' if x in low_freq_categories else x)

# Check the new value counts
print(application_df['ORGANIZATION'].value_counts())

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64


In [15]:
# Look at status value counts
print(application_df['STATUS'].value_counts())

1    34294
0        5
Name: STATUS, dtype: int64


In [16]:
# Calculate the correlation between STATUS and IS_SUCCESSFUL
corr = application_df[["STATUS", "IS_SUCCESSFUL"]].corr()

# Print the correlation matrix
print(corr)

                 STATUS  IS_SUCCESSFUL
STATUS         1.000000      -0.001636
IS_SUCCESSFUL -0.001636       1.000000


In [17]:
# Drop the STATUS Column
application_df = application_df.drop(['STATUS'], axis=1)

In [18]:
# Look at INCOME_AMT value counts for binning
print(application_df['INCOME_AMT'].value_counts())

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64


In [19]:
# Convert the INCOME_AMT column to a categorical variable
income_categories = ['0', '1M-5M', '100000-499999', '10M-50M', '25000-99999', '5M-10M', '10000-24999', '50M+']
application_df['INCOME_AMT'] = pd.Categorical(application_df['INCOME_AMT'], categories=income_categories, ordered=True)
# Replace missing values with the mode
income_mode = application_df['INCOME_AMT'].mode()[0]
application_df['INCOME_AMT'].replace(['1/01/9999'], income_mode, inplace=True)
# One-hot encode the INCOME_AMT column
income_dummies = pd.get_dummies(application_df['INCOME_AMT'], prefix='INCOME')
application_df = pd.concat([application_df, income_dummies], axis=1)

# Drop the original INCOME_AMT column
application_df.drop(['INCOME_AMT'], axis=1, inplace=True)

In [20]:
# Check the value counts of the new columns
print(application_df.filter(regex='^INCOME_').sum())

INCOME_0                24388
INCOME_1M-5M              955
INCOME_100000-499999     3374
INCOME_10M-50M            240
INCOME_25000-99999       3747
INCOME_5M-10M             185
INCOME_10000-24999        543
INCOME_50M+               139
dtype: int64


In [21]:
# Check the SPECIAL_CONSIDERATIONS
print(application_df['SPECIAL_CONSIDERATIONS'].value_counts())

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64


In [22]:
# Convert 'Y' to 1 and everything else to 0
application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].apply(lambda x: 1 if x == 'Y' else 0)

# Calculate the correlation coefficient
corr = application_df['IS_SUCCESSFUL'].corr(application_df['SPECIAL_CONSIDERATIONS'])

# Print the correlation coefficient
print(corr)

0.005469242791038586


In [23]:
# Drop the SPECIAL_CONSIDERATIONS Column
application_df = application_df.drop(['SPECIAL_CONSIDERATIONS'], axis=1)

In [24]:
# Look at ASK_AMT value counts for binning
print(application_df['ASK_AMT'].value_counts())

5000       25398
10478          3
15583          3
6725           3
63981          3
           ...  
772556         1
70103          1
27096          1
25049          1
1138700        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [25]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Reshape the feature to a 2D array and fit the scaler to the data
ask_amt = application_df["ASK_AMT"].values.reshape(-1, 1)
scaler.fit(ask_amt)

# Transform the feature using the scaler
scaled_ask_amt = scaler.transform(ask_amt)

# Replace the original feature in the DataFrame with the scaled feature
application_df["ASK_AMT"] = scaled_ask_amt

# View the updated DataFrame with the scaled feature
print(application_df.head())

  APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0            Other       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000         Other   

  ORGANIZATION       ASK_AMT  IS_SUCCESSFUL  INCOME_0  INCOME_1M-5M  \
0  Association  0.000000e+00              1         1             0   
1        Other  1.204843e-05              1         0             0   
2  Association  0.000000e+00              0         1             0   
3        Trust  1.967945e-07              1         0             0   
4        Trust  1.600293e-05              1         0             0   

   INCOME_100000-499999  INCOME_10M-50M  INCOME_25000-99999  INCOME_5M-10M  \
0                     0               0                   0              0   
1 

In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['AFFILIATION','CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'APPLICATION_TYPE'])
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,INCOME_0,INCOME_1M-5M,INCOME_100000-499999,INCOME_10M-50M,INCOME_25000-99999,INCOME_5M-10M,INCOME_10000-24999,INCOME_50M+,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6
0,0.000000e+00,1,1,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
1,1.204843e-05,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,0.000000e+00,0,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,0
3,1.967945e-07,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
4,1.600293e-05,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [27]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [29]:
# Define the model - deep neural net
nn = tf.keras.models.Sequential()

# Add input layer
nn.add(tf.keras.layers.Dense(units=9, input_dim=30, activation="tanh"))

# Add hidden layers with dropout regularization
nn.add(tf.keras.layers.Dense(units=5, activation="tanh"))
nn.add(tf.keras.layers.Dropout(0.3))
nn.add(tf.keras.layers.Dense(units=7, activation="tanh"))
nn.add(tf.keras.layers.Dropout(0.2))
nn.add(tf.keras.layers.Dense(units=7, activation="tanh"))
nn.add(tf.keras.layers.Dropout(0.2))

# Add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Print the model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 279       
                                                                 
 dense_1 (Dense)             (None, 5)                 50        
                                                                 
 dropout (Dropout)           (None, 5)                 0         
                                                                 
 dense_2 (Dense)             (None, 7)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 7)                 0         
                                                                 
 dense_3 (Dense)             (None, 7)                 56        
                                                                 
 dropout_2 (Dropout)         (None, 7)                 0

In [30]:
# Define the callback function
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="Models/model_weights_{epoch:02d}.h5", # Filepath to save the weights
    save_weights_only=True, # Save only the weights, not the entire model
    save_freq='epoch', # Save every epoch
    period=5 # Save every five epochs
)

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model
batch_size = 64
epochs = 200
nn.fit(X_train_scaled, y_train, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint_callback], verbose=1)

Epoch 1/200
402/402 [==============================] - 1s 1ms/step - loss: 0.6315 - accuracy: 0.6602
Epoch 2/200
402/402 [==============================] - 0s 1ms/step - loss: 0.5982 - accuracy: 0.7142
Epoch 3/200
402/402 [==============================] - 0s 1ms/step - loss: 0.5919 - accuracy: 0.7170
Epoch 4/200
402/402 [==============================] - 0s 1ms/step - loss: 0.5873 - accuracy: 0.7214
Epoch 5/200
402/402 [==============================] - 0s 1ms/step - loss: 0.5852 - accuracy: 0.7241
Epoch 6/200
402/402 [==============================] - 0s 1ms/step - loss: 0.5830 - accuracy: 0.7250
Epoch 7/200
402/402 [==============================] - 0s 1ms/step - loss: 0.5806 - accuracy: 0.7262
Epoch 8/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5802 - accuracy: 0.7255
Epoch 9/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5763 - accuracy: 0.7278
Epoch 10/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5775 - accura

402/402 [==============================] - 1s 3ms/step - loss: 0.5621 - accuracy: 0.7321
Epoch 81/200
402/402 [==============================] - 2s 5ms/step - loss: 0.5630 - accuracy: 0.7332
Epoch 82/200
402/402 [==============================] - 2s 4ms/step - loss: 0.5627 - accuracy: 0.7338
Epoch 83/200
402/402 [==============================] - 2s 4ms/step - loss: 0.5627 - accuracy: 0.7341
Epoch 84/200
402/402 [==============================] - 2s 5ms/step - loss: 0.5622 - accuracy: 0.7332
Epoch 85/200
402/402 [==============================] - 2s 6ms/step - loss: 0.5631 - accuracy: 0.7332
Epoch 86/200
402/402 [==============================] - 2s 5ms/step - loss: 0.5632 - accuracy: 0.7334
Epoch 87/200
402/402 [==============================] - 2s 5ms/step - loss: 0.5641 - accuracy: 0.7327
Epoch 88/200
402/402 [==============================] - 2s 5ms/step - loss: 0.5619 - accuracy: 0.7327
Epoch 89/200
402/402 [==============================] - 2s 5ms/step - loss: 0.5630 - accuracy: 

402/402 [==============================] - 1s 1ms/step - loss: 0.5604 - accuracy: 0.7355
Epoch 160/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5603 - accuracy: 0.7341
Epoch 161/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5611 - accuracy: 0.7346
Epoch 162/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5602 - accuracy: 0.7339
Epoch 163/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5601 - accuracy: 0.7348
Epoch 164/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5592 - accuracy: 0.7360
Epoch 165/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5613 - accuracy: 0.7340
Epoch 166/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5597 - accuracy: 0.7346
Epoch 167/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5613 - accuracy: 0.7334
Epoch 168/200
402/402 [==============================] - 1s 1ms/step - loss: 0.5595 - a

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5606 - accuracy: 0.7257 - 366ms/epoch - 1ms/step
Loss: 0.5605912804603577, Accuracy: 0.7257142663002014


In [32]:
# Export our model to HDF5 file
nn.save("Models/AlphabetSoupCharity.h5")